# Training ResNet and SparseResNet on clean images 
### Flatten ReLU

In [1]:
import sys, os
d = os.path.dirname(os.getcwd())
sys.path.insert(0, d)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader

from utils.train_utils import train_model
from models.resnet_FReLU import FResNet, FSparseResNet
from utils.attacks import pgd

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [4]:
tr_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
vl_transform = transforms.Compose([
    transforms.ToTensor()
])

ds = CIFAR10('../data', train=True, download=True, transform=tr_transform)
ds_test = CIFAR10('../data', train=False, download=True, transform=vl_transform)

batch_size = 16
train_dl = DataLoader(ds, batch_size, shuffle=True)
valid_dl = DataLoader(ds_test, batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


# Training on clean images

### ResNet

In [5]:
model = FResNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', min_lr=1e-7, patience=2, verbose=True)

train_model(model, train_dl, valid_dl, optimizer, loss_fn, scheduler, epochs=12, device=device)
torch.save(model.state_dict(), "../saved/resnet_DTL.pt")

Epoch: 1 Validation Loss: 1.1052 accuracy: 0.5970, time: 0:15:15
Epoch: 2 Validation Loss: 0.8592 accuracy: 0.6948, time: 0:15:17
Epoch: 3 Validation Loss: 0.6882 accuracy: 0.7583, time: 0:15:20
Epoch: 4 Validation Loss: 0.6115 accuracy: 0.7857, time: 0:15:19
Epoch: 5 Validation Loss: 0.5860 accuracy: 0.8033, time: 0:15:20
Epoch: 6 Validation Loss: 0.4678 accuracy: 0.8417, time: 0:15:18
Epoch: 7 Validation Loss: 0.4677 accuracy: 0.8419, time: 0:15:19
Epoch: 8 Validation Loss: 0.4997 accuracy: 0.8373, time: 0:15:18
epochs_no_improve: 1/4
Epoch: 9 Validation Loss: 0.4273 accuracy: 0.8579, time: 0:15:17
Epoch: 10 Validation Loss: 0.3947 accuracy: 0.8681, time: 0:15:19
Epoch: 11 Validation Loss: 0.3797 accuracy: 0.8724, time: 0:15:20
Epoch: 12 Validation Loss: 0.3833 accuracy: 0.8715, time: 0:15:20
epochs_no_improve: 1/4


### SparseResNet

In [5]:
model = FSparseResNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', min_lr=1e-7, patience=2, verbose=True)

train_model(model, train_dl, valid_dl, optimizer, loss_fn, scheduler, epochs=7, sparse=True, device=device)
torch.save(model.state_dict(), "../saved/sparse_resnet_DTL.pt")

Epoch: 1 Validation Loss: 1.3578 accuracy: 0.4939, time: 0:17:27
Epoch: 2 Validation Loss: 0.9670 accuracy: 0.6531, time: 0:17:20
Epoch: 3 Validation Loss: 0.8113 accuracy: 0.7167, time: 0:17:23
Epoch: 4 Validation Loss: 0.7423 accuracy: 0.7380, time: 0:17:31
Epoch: 5 Validation Loss: 0.6602 accuracy: 0.7723, time: 0:17:41
Epoch: 6 Validation Loss: 0.6131 accuracy: 0.7891, time: 0:17:32
Epoch: 7 Validation Loss: 0.5638 accuracy: 0.8051, time: 0:17:19
